In [3]:
OPENAI_KEY = ""

# Transcribing, Translating, and Adding English Subtitle to a Dutch Video

1. Video format check/conversion (.webm to .mp4) "what the fuck is .wedm?" (library ffmpeg)
2. Video-to-audio (.webm to .mp3) (library moviepy)
3. Audio-to-text (.str output) (library whisper)
4. Text translation (.str output) (Azure OpenAI gpt4)
5. MANUALLY checking the result by playing it using VLC (and manually fixing mistakes)
6. Improving the subtitle text style (converting from .srt to .ass)
8. Adding the translated subtitle to the video 

# .webm to .mp4 (video + audio)

In [ ]:
# !pip install ffmpeg-python

In [ ]:
import ffmpeg

# Path to the input .webm file
input_file = "Microsoft PotY speech for Northwave.webm"

# Path to the output .mp4 file
output_file = "Microsoft PotY speech for Northwave.mp4"

# Perform the conversion
(
    ffmpeg
    .input(input_file)
    .output(output_file, vcodec='libx264', acodec='aac')
    .run(overwrite_output=True)
)

# Video-to-audio (.webm to .mp3)

In [ ]:
# !pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

input_file = "Microsoft PotY speech for Northwave.webm"
output_file = "Microsoft PotY speech for Northwave.mp3"

video = VideoFileClip(input_file)
audio = video.audio
audio.write_audiofile(output_file)

# text-to-speech

In [ ]:
from gtts import gTTS
import playsound
import os

def text2speech(text,lang):
    tts = gTTS(text=text, lang=lang)
    filename = "temp.mp3"
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)

In [ ]:
text = "Goedemorgen. Het testen van de tekst naar spraak en de spraak naar transcriptie."
lang = "nl"
text2speech(text,lang)

# text-to-speech_mp3

In [ ]:
from gtts import gTTS

def text2mp3(text, lang, output_filename="output.mp3"):
    tts = gTTS(text=text, lang=lang)
    tts.save(output_filename)

In [ ]:
text = "Goedemorgen. Het testen van de tekst naar spraak en de spraak naar transcriptie."
lang = 'nl'
output_filename = "output.mp3"
text2mp3(text, lang, output_filename="output.mp3")

# speech-to-text

In [ ]:
# !pip install git+https://github.com/openai/whisper.git

In [ ]:
import whisper
model = whisper.load_model("base")

In [ ]:
audio_path = "output.mp3"
result = model.transcribe(audio_path, language='nl')
result['text']

# audio-to-text .srt

In [ ]:
import whisper
import datetime

# Load the Whisper model
model = whisper.load_model("base")

# Path to the audio file
audio_path = "Microsoft PotY speech for Northwave.mp3"

# Transcribe the audio file
result = model.transcribe(audio_path, language='nl')

# Function to format time in SRT time format
def format_timestamp(seconds):
    ms = int((seconds % 1) * 1000)
    seconds = int(seconds)
    hrs = seconds // 3600
    mins = (seconds % 3600) // 60
    secs = seconds % 60
    return f"{hrs:02}:{mins:02}:{secs:02},{ms:03}"

# Open a file to write the SRT output
with open("output.srt", "w", encoding="utf-8") as srt_file:
    for i, segment in enumerate(result['segments']):
        start_time = format_timestamp(segment['start'])
        end_time = format_timestamp(segment['end'])
        text = segment['text']
        
        # Write the SRT entry
        srt_file.write(f"{i + 1}\n")
        srt_file.write(f"{start_time} --> {end_time}\n")
        srt_file.write(f"{text}\n\n")


# Convert an .srt file from Dutch to English

In [ ]:
# !pip install openai

In [ ]:
import os
from openai import AzureOpenAI
import time

def azureopenai_gpt4_1106(OPENAI_KEY, system_prompt, input_message):
    client = AzureOpenAI(azure_endpoint="https://instance-openai-france.openai.azure.com/",
                         api_key=OPENAI_KEY,  
                         api_version="2024-02-15-preview")
    
    try:
        response = client.chat.completions.create(
            model="gpt4-1106",  # model = "deployment_name"
            messages=[{"role": "system", "content": system_prompt},
                      {"role": "user", "content": input_message}]
        )
        output = response.choices[0].message.content
    except Exception as e:
        output = {'error': str(e)}

    return output

In [ ]:
def translate_srt_file(input_srt_path, output_srt_path, openai_key):
    # Read the .srt file
    with open(input_srt_path, 'r', encoding='utf-8') as file:
        srt_content = file.read()

    # Define the system prompt for translation
    system_prompt = "Translate the following Dutch subtitles to English:"

    # Translate the .srt content
    response = azureopenai_gpt4_1106(openai_key, system_prompt, srt_content)

    if 'error' in response:
        print(f"Error: {response['error']}")
        return
    
    translated_content = response

    # Write the translated content to a new .srt file
    with open(output_srt_path, 'w', encoding='utf-8') as file:
        file.write(translated_content)

    print(f"Translated .srt file has been saved as {output_srt_path}")

In [ ]:
input_srt_path = "Microsoft PotY speech for Northwave_nl.srt"
output_srt_path = "Microsoft PotY speech for Northwave_en.srt"

translate_srt_file(input_srt_path, output_srt_path, OPENAI_KEY)

# Convert from .srt to .ass (for a better subtitle formating)

In [ ]:
!pip install pysubs2

In [ ]:
import pysubs2

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('&H').lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def convert_srt_to_ass(srt_path, ass_path, font_name="Roboto", font_size=24, primary_color="&H00FFFFFF", outline_color="&H00000000", back_color="&H64000000"):
    subs = pysubs2.load(srt_path, encoding="utf-8")
    
    # Define styles
    style = pysubs2.SSAStyle()
    style.fontname = font_name
    style.fontsize = font_size
    style.primarycolor = pysubs2.Color(*hex_to_rgb(primary_color))
    style.outlinecolor = pysubs2.Color(*hex_to_rgb(outline_color))
    style.backcolor = pysubs2.Color(*hex_to_rgb(back_color))
    style.bold = -1
    style.shadow = 2
    style.outline = 1
    style.alignment = pysubs2.Alignment.BOTTOM_CENTER
    style.marginl = 10
    style.marginr = 10
    style.marginv = 10

    subs.styles["Default"] = style
    
    subs.save(ass_path)


In [ ]:
# Example usage
srt_path = 'Microsoft PotY speech for Northwave_en_v3.srt'
ass_path = 'Microsoft PotY speech for Northwave_en_v3.ass'

convert_srt_to_ass(srt_path, ass_path)

# Add subtitle to video

In [ ]:
import ffmpeg

def add_subtitle(video_path, subtitle_path, output_path):
    (
        ffmpeg
        .input(video_path)
        .output(output_path, vf=f"subtitles={subtitle_path}")
        .run(overwrite_output=True)
    )

In [ ]:
# Example usage
video_path = 'Microsoft PotY speech for Northwave.mp4'
subtitle_path = 'Microsoft PotY speech for Northwave_en_v3.ass'
output_path = 'Microsoft PotY speech for Northwave_EN_v2.mp4'

add_subtitle(video_path, subtitle_path, output_path)

# MERGING EVERYTHING!!!!

In [6]:
import ffmpeg
from moviepy.editor import VideoFileClip
import whisper
import datetime
from openai import AzureOpenAI
import time
import pysubs2

In [7]:
input_file = "Microsoft PotY speech for Northwave.webm"

In [8]:
### Converting .webm to .mp4 (video to video)
print("- Converting .webm to .mp4 (video to video)")
video_mp4 = input_file.split('.')[0]+'.mp4'
(
    ffmpeg
    .input(input_file)
    .output(video_mp4, vcodec='libx264', acodec='aac')
    .run(overwrite_output=True, quiet=True)
)

### Extracting only the audio from the original video
print("- Extracting only the audio from the original video")
audio_path = input_file.split('.')[0]+'.mp3'
video = VideoFileClip(input_file)
audio = video.audio
audio.write_audiofile(audio_path, verbose=False, logger=None)

### Converting speech-to-text (in the original format, Dutch)
print("- Converting speech-to-text (in the original format, Dutch)")
model = whisper.load_model("base")
result = model.transcribe(audio_path, language='nl')

# Function to format time in SRT time format
def format_timestamp(seconds):
    ms = int((seconds % 1) * 1000)
    seconds = int(seconds)
    hrs = seconds // 3600
    mins = (seconds % 3600) // 60
    secs = seconds % 60
    return f"{hrs:02}:{mins:02}:{secs:02},{ms:03}"

subtitle_nl = input_file.split('.')[0]+'_NL.str'
with open(subtitle_nl, "w", encoding="utf-8") as srt_file:
    for i, segment in enumerate(result['segments']):
        start_time = format_timestamp(segment['start'])
        end_time = format_timestamp(segment['end'])
        text = segment['text']
        
        # Write the SRT entry
        srt_file.write(f"{i + 1}\n")
        srt_file.write(f"{start_time} --> {end_time}\n")
        srt_file.write(f"{text}\n\n")

### Translating the text in the .str from Dutch to English using Azure OpenAI
print("- Translating the text in the .str from Dutch to English using Azure OpenAI")
def azureopenai_gpt4_1106(OPENAI_KEY, system_prompt, input_message):
    client = AzureOpenAI(azure_endpoint="https://instance-openai-france.openai.azure.com/",
                         api_key=OPENAI_KEY,  
                         api_version="2024-02-15-preview")
    
    try:
        response = client.chat.completions.create(
            model="gpt4-1106",  # model = "deployment_name"
            messages=[{"role": "system", "content": system_prompt},
                      {"role": "user", "content": input_message}]
        )
        output = response.choices[0].message.content
    except Exception as e:
        output = {'error': str(e)}

    return output

def translate_srt_file(input_srt_path, output_srt_path, openai_key):
    # Read the .srt file
    with open(input_srt_path, 'r', encoding='utf-8') as file:
        srt_content = file.read()

    # Define the system prompt for translation
    system_prompt = "Translate the following Dutch subtitles to English:"

    # Translate the .srt content
    response = azureopenai_gpt4_1106(openai_key, system_prompt, srt_content)

    if 'error' in response:
        print(f"Error: {response['error']}")
        return
    
    translated_content = response

    # Write the translated content to a new .srt file
    with open(output_srt_path, 'w', encoding='utf-8') as file:
        file.write(translated_content)

subtitle_en = input_file.split('.')[0]+'_EN.str'
translate_srt_file(subtitle_nl, subtitle_en, OPENAI_KEY)

################
### MANUAL STEP TO VALIDATE AND FIX THE translated to English subtitle
###############

### Converting the .str to .ass for a better formating of the subtitle
print("- Converting the .str to .ass for a better formating of the subtitle")
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('&H').lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def convert_srt_to_ass(srt_path, ass_path, font_name="Roboto", font_size=24, primary_color="&H00FFFFFF", outline_color="&H00000000", back_color="&H64000000"):
    subs = pysubs2.load(srt_path, encoding="utf-8")
    
    # Define styles
    style = pysubs2.SSAStyle()
    style.fontname = font_name
    style.fontsize = font_size
    style.primarycolor = pysubs2.Color(*hex_to_rgb(primary_color))
    style.outlinecolor = pysubs2.Color(*hex_to_rgb(outline_color))
    style.backcolor = pysubs2.Color(*hex_to_rgb(back_color))
    style.bold = -1
    style.shadow = 2
    style.outline = 1
    style.alignment = pysubs2.Alignment.BOTTOM_CENTER
    style.marginl = 10
    style.marginr = 10
    style.marginv = 10

    subs.styles["Default"] = style
    
    subs.save(ass_path)

subtitle_en_ass = input_file.split('.')[0]+'_EN.ass'

convert_srt_to_ass(subtitle_en, subtitle_en_ass)

### Adding the final translated subtitle with nice style to video
print("- Adding the final translated subtitle with nice style to video")
def add_subtitle(video_path, subtitle_path, output_path):
    (
        ffmpeg
        .input(video_path)
        .output(output_path, vf=f"subtitles={subtitle_path}")
        .run(overwrite_output=True, quiet=True)
    )

final_video = input_file.split('.')[0]+'_EN.mp4'
add_subtitle(video_mp4, subtitle_en_ass, final_video)

print("CONGRATULATIONS!")

- Converting .webm to .mp4 (video to video)
- Extracting only the audio from the original video
- Converting speech-to-text (in the original format, Dutch)
- Translating the text in the .str from Dutch to English using Azure OpenAI
- Converting the .str to .ass for a better formating of the subtitle
- Adding the final translated subtitle with nice style to video


# In case there is a better version of the .srt

In [10]:
subtitle_en_improved = "Microsoft PotY speech for Northwave_EN_v2.str"

### Convert from .str to .ass
subtitle_en_ass = subtitle_en_improved.split('.')[0]+'.ass'
convert_srt_to_ass(subtitle_en_improved, subtitle_en_ass)

### Adding the final translated subtitle with nice style to video
final_video = subtitle_en_improved.split('.')[0]+'.mp4'
add_subtitle(video_mp4, subtitle_en_ass, final_video)

print("CONGRATULATIONS!")

CONGRATULATIONS!
